<a href="https://colab.research.google.com/drive/1p7CfHfWGj61rBE8U_uih-IIW98kr3nuw?usp=sharing" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>

### Automatic Reasoning and Tool-Use (ART)

ART is an advanced prompting method that combines reasoning with automated tool selection and use.

### Key points:

* **Core concept:** Enable language models to autonomously reason about problems and select/use appropriate tools.

* **Components:**
  * Reasoning module
  * Tool selection mechanism
  * Tool use interface

* **Process:**
  * Analyze the problem through reasoning
  * Identify and select relevant tools
  * Use tools to gather information or perform actions
  * Integrate tool outputs into the reasoning process

* **Advantages:**
  * Enhances problem-solving capabilities
  * Allows for more complex, multi-step tasks

* **Applications:**
  * Data analysis
  * Web-based research
  * Complex decision-making scenarios

* **Implementation:**
  * Define a set of available tools and their functions
  * Design prompts that encourage tool consideration
  * Implement feedback loops between reasoning and tool use

* **Challenges:**
  * Ensuring appropriate tool selection
  * Managing context across multiple tool uses

* **Example structure:**
  * Thought: [Reasoning about the problem]
  * Tool Selection: [Choose appropriate tool]
  * Tool Use: [Apply selected tool]
  * Integration: [Incorporate tool output into reasoning]

In [1]:
!pip install -q -U langchain-groq==0.2.4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 3.7 MB/s eta 0:00:00


In [6]:
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate
from IPython.display import display, Markdown
from datetime import datetime, date
import json

In [3]:
import os
import getpass

### 🔑 Provide Groq API Key

- [Groq API Key](https://console.groq.com/keys)


In [4]:
os.environ["GROQ_API_KEY"] = getpass.getpass()

··········


In [5]:
llm = ChatGroq(
    model="llama3-8b-8192",
    temperature=0.5
)

In [11]:
# Define available tools
def calculate_days(date_string):
    target_date = datetime.strptime(date_string, "%Y-%m-%d").date()
    current_date = date.today()
    return (target_date - current_date).days

tools = {
    "calculator": lambda x: eval(x),
    "date": lambda: date.today().strftime("%Y-%m-%d"),
    "weather": lambda city: f"The weather in {city} is sunny with a high of 25°C.",
    "days_between": calculate_days
}

# ART Prompt
art_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", """You are an AI assistant capable of breaking down complex tasks, identifying necessary tools, and applying them to solve problems. You have access to the following tools:
          1. calculator: Performs mathematical calculations. Input should be a mathematical expression.
          2. date: Returns the current date.
          3. weather: Provides weather information for a given city.
          4. days_between: Calculates the number of days between the current date and a given date (format: YYYY-MM-DD).
          For each step in your reasoning, if a tool is needed, specify it in the following JSON format:
          {{"tool": "tool_name", "input": "tool_input"}}
          Your final answer should not be in JSON format."""),
        (
            "human", """Task: {task}
              Break down this task and solve it step by step. For each step, explain your reasoning and use tools when necessary.
              Your response:"""
        ),
    ]
)

def art_response(task):
    chain = art_prompt | llm
    raw_response = chain.invoke({"task": task}).content
    # Process the response to use tools
    lines = raw_response.split("\n")
    processed_response = []

    for line in lines:
        if line.strip().startswith("{") and line.strip().endswith("}"):
            try:
                tool_call = json.loads(line.strip())
                if "tool" in tool_call and "input" in tool_call:
                    if tool_call["tool"] in tools:
                        # Handle different tools properly
                        if tool_call["tool"] == "date":
                            tool_result = tools[tool_call["tool"]]()
                        else:
                            tool_result = tools[tool_call["tool"]](tool_call["input"])

                        processed_response.append(
                            f"Using {tool_call['tool']}: {tool_call['input']}"
                        )
                        processed_response.append(f"Result: {tool_result}")
                    else:
                        processed_response.append(
                            f"Error: Tool '{tool_call['tool']}' not found."
                        )
                else:
                    processed_response.append(line)
            except json.JSONDecodeError:
                processed_response.append(line)
        else:
            processed_response.append(line)

    return "\n".join(processed_response)



In [12]:
# Example usage
task = "Calculate the number of days between the current date and July 4, 2025. Then, provide the weather forecast for New York City."
response = art_response(task)
display(
    Markdown(
        f"""**Automatic Reasoning and Tool-Use (ART) Response:**
Task: {task}
{response}"""
    )
)

**Automatic Reasoning and Tool-Use (ART) Response:**
Task: Calculate the number of days between the current date and July 4, 2025. Then, provide the weather forecast for New York City.
Let's break down the task into smaller steps and solve it step by step.

Step 1: Calculate the number of days between the current date and July 4, 2025.

To do this, we need to calculate the difference between the current date and July 4, 2025. We can use the "date" tool to get the current date, and then use the "days_between" tool to calculate the number of days between the current date and July 4, 2025.

Using date: 
Result: 2025-02-25

This will give us the current date. Let's assume the current date is 2023-09-01.

Next, we can use the "days_between" tool to calculate the number of days between the current date (2023-09-01) and July 4, 2025.

Using days_between: 2025-07-04
Result: 129

This will give us the number of days between the current date and July 4, 2025.

Step 2: Provide the weather forecast for New York City.

To do this, we can use the "weather" tool and input the city name as "New York City".

Using weather: New York City
Result: The weather in New York City is sunny with a high of 25°C.

This will give us the current weather forecast for New York City.

In conclusion, the number of days between the current date (2023-09-01) and July 4, 2025 is 1091 days. The current weather forecast for New York City is [insert weather forecast here].